# Object Tracking Algorithm Tutorial

This notebook present all the tracking techniques available with opencv and how to use them.

## Virtual environnement
Creation virtual environnement and install package to use jupyter notebook:
```console
python -m venv env
source ./env/bin/activate
python -m pip install --upgrade pip
pip install ipykernel
python -m ipykernel install --user --name=env
```
Then one the jupyter notebook select the correct kernel.

## Install Dependencies

In [ ]:
!pip install opencv-python opencv-contrib-python matplotlib

## Import Packages

In [2]:
import os
import sys
import time
from random import randint
import numpy as np

import cv2
import matplotlib.pyplot as plt

## Create Folders

In [3]:
DIR_NAME = os.getcwd()
paths = {
    'DATA_PATH' : os.path.join(DIR_NAME,'data'),
    'VIDEOS_PATH' : os.path.join(DIR_NAME,'data','videos'),
    'CASCADE_PATH' : os.path.join(DIR_NAME,'data','cascade')  # only if you have a cascade file
}

In [4]:
files = {
    'GOTURN_CAFFEMODEL' : os.path.join(paths['DATA_PATH'],'goturn.caffemodel'),
    'GOTURN_PROTOTXT' : os.path.join(paths['DATA_PATH'],'goturn.prototxt'),
}

In [5]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

## Create Tracker

In [6]:
tracker_types = ['BOOSTING', 'MIL', 'KCF', 'TLD', 'MEDIANFLOW', 'MOSSE', 'CSRT']

In [7]:
def create_tracker_by_name(tracker_type):
    if tracker_type == tracker_types[0]:
        tracker = cv2.legacy.TrackerBoosting_create()
    elif tracker_type == tracker_types[1]:
        tracker = cv2.legacy.TrackerMIL_create()
    elif tracker_type == tracker_types[2]:
        tracker = cv2.legacy.TrackerKCF_create()
    elif tracker_type == tracker_types[3]:
        tracker = cv2.legacy.TrackerTLD_create()
    elif tracker_type == tracker_types[4]:
        tracker = cv2.legacy.TrackerMedianFlow_create()
    elif tracker_type == tracker_types[5]:
        tracker = cv2.legacy.TrackerMOSSE_create()
    elif tracker_type == tracker_types[6]:
        tracker = cv2.legacy.TrackerCSRT_create()
    else:
        tracker = None
        print('Invalid name! Available trackers: ')
        for t in tracker_types:
            print(t)
    return tracker

## Single Tracking

In [9]:
tracker_type = 'CSRT'

In [10]:
tracker = create_tracker_by_name(tracker_type)

In [11]:
video_name = 'race.mp4'

In [12]:
# open video
video = cv2.VideoCapture(os.path.join(paths['VIDEOS_PATH'],video_name))
if not video.isOpened():
    print('Error while loading the video!')
    sys.exit()
flag, frame = video.read()
if not flag:
    print('Erro while loading the frame!')
    sys.exit()

In [13]:
bbox = cv2.selectROI(frame) # region of interest
cv2.destroyAllWindows()
flag = tracker.init(frame, bbox)
if not flag:
    print('Error while initializing tracker.')
colors = (randint(0, 255), randint(0,255), randint(0, 255)) # RGB -> BGR

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


In [14]:
prev_frame_time, new_frame_time = 0, 0
while True:
    ok, frame = video.read()
    #print(ok)
    if not ok:
        break

    ok, bbox = tracker.update(frame)
    #print(ok, bbox)
    if ok == True:
        (x, y, w, h) = [int(v) for v in bbox]
        #print(x, y, w, h)
        cv2.rectangle(frame, (x, y), (x + w, y + h), colors, 2)
    else:
        cv2.putText(frame, 'Tracking failure!', (100,80), cv2.FONT_HERSHEY_SIMPLEX, .75, (0,0,255))

    new_frame_time = time.time()
    fps = round(1/(new_frame_time-prev_frame_time),2)
    prev_frame_time = new_frame_time
    cv2.putText(frame,f'Tracker : {tracker_type}, FPS : {fps}HZ', (100, 20), cv2.FONT_HERSHEY_SIMPLEX, .75, (0, 0, 255))

    cv2.imshow('Tracking', frame)
    if cv2.waitKey(1) & 0XFF == 27: # esc
        break
    
cv2.destroyAllWindows()

## Multi Tracking

In [15]:
video_name = 'race.mp4'

In [16]:
# open video
video = cv2.VideoCapture(os.path.join(paths['VIDEOS_PATH'],video_name))
if not video.isOpened():
    print('Error while loading the video!')
    sys.exit()
flag, frame = video.read()
if not flag:
    print('Erro while loading the frame!')
    sys.exit()

In [17]:
# define bounding boxes to track
bboxes = []
colors = []
while True:
    bbox = cv2.selectROI('MultiTracker', frame)
    bboxes.append(bbox)
    colors.append((randint(0,255), randint(0,255), randint(0,255)))
    print('Press Q to quit and start tracking')
    print('Press any other key to select the next object')
    k = cv2.waitKey(0) & 0XFF
    if k == 113: # Q - quit
        break
cv2.destroyAllWindows()

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Press Q to quit and start tracking
Press any other key to select the next object
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Press Q to quit and start tracking
Press any other key to select the next object
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Press Q to quit and start tracking
Press any other key to select the next object
Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Press Q to quit and start tracking
Press any other key to select the next object


In [18]:
tracker_type = 'CSRT'

In [19]:
# init tracker
multi_tracker = cv2.legacy.MultiTracker_create()
for bbox in bboxes:
    multi_tracker.add(create_tracker_by_name(tracker_type), frame, bbox)

In [20]:
# perform tracking
prev_frame_time, new_frame_time = 0, 0
while video.isOpened():
    flag, frame = video.read()
    if not flag:
        break

    flag, boxes = multi_tracker.update(frame)

    if flag : 
        for i, box in enumerate(boxes):
            (x, y, w, h) = [int(v) for v in box]
            cv2.rectangle(frame, (x, y), (x + w, y + h), colors[i], 2)
    else :
        cv2.putText(frame, 'Tracking failure!', (100,80), cv2.FONT_HERSHEY_SIMPLEX, .75, (0,0,255))
 
    new_frame_time = time.time()
    fps = round(1/(new_frame_time-prev_frame_time),2)
    prev_frame_time = new_frame_time
    cv2.putText(frame,f'Tracker : {tracker_type}, FPS : {fps}HZ', (100, 20), cv2.FONT_HERSHEY_SIMPLEX, .75, (0, 0, 255))

    cv2.imshow('MultiTracker', frame)
    if cv2.waitKey(1) & 0XFF == 27: # esc
        break
cv2.destroyAllWindows()

## Goturn Tracking

Use deep learning model, need two file 
- [coffemodel](./goturn.caffemodel) : contains the weight
- [prototxt](./goturn.prototxt) : contains the data

To be sure to get good result, you need to be sure the object you track is in the training dataset of the model because it is an offline model.

Go to this [repository](https://github.com/spmallick/goturn-files.git) to download examples.

In [8]:
if not (os.path.isfile(files['GOTURN_CAFFEMODEL']) and os.path.isfile(files['GOTURN_PROTOTXT'])):
    print('Error loading the network files!')
    sys.exit()

In [10]:
# create tracker
tracker = cv2.TrackerGOTURN_create()

In [11]:
video_name = 'race.mp4'

In [12]:
# load video and first frame
video = cv2.VideoCapture(os.path.join(paths['VIDEOS_PATH'],video_name))
if not video.isOpened():
    print('Error while loading the video!')
    sys.exit()
flag, frame = video.read()
if not flag:
    print('Erro while loading the frame!')
    sys.exit()


In [13]:
# init bounding box and tracker
bbox = cv2.selectROI(frame) # region of interest
cv2.destroyAllWindows()
colors = (randint(0, 255), randint(0, 255), randint(0, 255))
flag = tracker.init(frame, bbox)

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


In [14]:
# perform tracking
prev_frame_time = 0
new_frame_time = 0
while True:
    flag, frame = video.read()
    if not flag:
        break

    flag, bbox = tracker.update(frame)
    if flag == True:
        (x, y, w, h) = [int(v) for v in bbox]
        cv2.rectangle(frame, (x, y), (x + w, y + h), colors, 2)
    else:
        cv2.putText(frame, 'Tracking failure!', (100,80), cv2.FONT_HERSHEY_SIMPLEX, .75, (0,0,255))

    new_frame_time = time.time()
    fps = round(1/(new_frame_time-prev_frame_time),2)
    prev_frame_time = new_frame_time
    cv2.putText(frame,f'Tracker : GOTURN, FPS : {fps}HZ', (100, 20), cv2.FONT_HERSHEY_SIMPLEX, .75, (0, 0, 255))

    cv2.imshow('Tracking', frame)
    if cv2.waitKey(1) & 0XFF == 27: # esc
        break
cv2.destroyAllWindows()

## Object Detection

To improve you tracking, you can use object detection. If your tracker tell you when you have lost you the object, you can perform object detection

In [ ]:
tracker_type = 'CRST'

In [ ]:
video_name = 'walking.avi'

In [ ]:
# init video and tracker
tracker = create_tracker_by_name(tracker_type)
video = cv2.VideoCapture(os.path.join(paths['VIDEOS_PATH'],video_name))
if not video.isOpened():
    print('Error while loading the video!')
    sys.exit()
flag, frame = video.read()
if not flag:
    print('Erro while loading the frame!')
    sys.exit()

In [ ]:
# init detector
cascade_name = 'fullbody.xml'
detector = cv2.CascadeClassifier(os.path.join(paths['CASCADE_PATH'],cascade_name))

In [ ]:
def detect():
    while True:
        flag, frame = video.read()
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # change function if detector change
        detections = detector.detectMultiScale(frame_gray, minSize=(60,60)) 
        for (x, y, w, h) in detections:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0,0,255), 2)
            cv2.imshow('Detection', frame)
            if x > 0:
                print('Haarscade detection')
                return x, y, w, h

In [ ]:
# init bounding box and tracking
bbox = detect()
flag = tracker.init(frame, bbox)
colors = (randint(0, 255), randint(0, 255), randint(0, 255))

In [ ]:
# perform tracking and detection
prev_frame_time = 0
new_frame_time = 0
while True:
    flag, frame = video.read()
    if not flag:
        break
    flag, bbox = tracker.update(frame)
    if flag:
        (x, y, w, h) = [int(v) for v in bbox]
        cv2.rectangle(frame, (x, y), (x + w, y + h), colors)
    else:
        print('Tracking failure! We will execute the haarcascade detector')
        bbox = detect()
        tracker = create_tracker_by_name(tracker_type)
        tracker.init(frame, bbox)

    new_frame_time = time.time()
    fps = round(1/(new_frame_time-prev_frame_time),2)
    prev_frame_time = new_frame_time
    cv2.putText(frame,f'Tracker : {tracker_type}, FPS : {fps}HZ', (100, 20), cv2.FONT_HERSHEY_SIMPLEX, .75, (0, 0, 255))

    cv2.imshow('Tracking', frame)
    k = cv2.waitKey(1) & 0XFF
    if k == 27: # esc
        break
cv2.destroyAllWindows()

## MeanShift

Useful links:
- [hsv color space](https://cran.r-project.org/web/packages/colordistance/vignettes/color-spaces.html)
- [opencv color space](https://www.learnopencv.com/color-spaces-in-opencv-cpp-python/)
- [opencv histogram](https://docs.opencv.org/master/d1/db7/tutorial_py_histogram_begins.html)
- [opencv back projection](https://docs.opencv.org/3.4.15/da/d7f/tutorial_back_projection.html)

In [25]:
# initialise camera and bounding box
cap = cv2.VideoCapture(0)
if not video.isOpened():
    print('Error while loading the video!')
    sys.exit()
time.sleep(1.5)
flag, frame = cap.read()
if not flag:
    print('Erro while loading the frame!')
    sys.exit()
bbox = cv2.selectROI(frame)
cv2.destroyAllWindows()
x, y, w, h = bbox
track_window = (x, y, w, h)

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!


In [ ]:
colors = (randint(0, 255), randint(0, 255), randint(0, 255))

In [ ]:
# compute hsv histogram
roi = frame[y:y+h, x:x+w] # RGB -> BGR
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0,180])

In [ ]:
# show histogram
plt.hist(roi.ravel(), 180, [0, 180])
plt.show()

In [ ]:
roi_hist = cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
# define (param1|param2...),value1,value2...
parameters = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

In [ ]:
# perform tracking
prev_frame_time = 0
new_frame_time = 0
while True:
    flag, frame = cap.read()
    if flag:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0,180], 1)
        flag, track_window = cv2.meanShift(dst, (x, y, w, h), parameters)
        if flag:
            x, y, w, h = track_window
            cv2.rectangle(frame, (x, y), (x + w, y + h), colors, 2)
        else:
            cv2.putText(frame, 'Tracking failure!', (100,80), cv2.FONT_HERSHEY_SIMPLEX, .75, (0,0,255))

        new_frame_time = time.time()
        fps = round(1/(new_frame_time-prev_frame_time),2)
        prev_frame_time = new_frame_time
        cv2.putText(frame,f'Tracker : MeanShift, FPS : {fps}HZ', (100, 20), cv2.FONT_HERSHEY_SIMPLEX, .75, (0, 0, 255))

        cv2.imshow('Meanshift tracking', frame)
        cv2.imshow('Meanshift distribution', dst)


        if cv2.waitKey(1) == 13: # esc
            break
    else:
        break

cv2.destroyAllWindows()
cap.release()

## CamShift

Update of MeanShift as it adapt the size of the bounding box.
Useful Links : 
- [opencv polylines](https://docs.opencv.org/4.5.3/d6/d6e/group__imgproc__draw.html#ga1ea127ffbbb7e0bfc4fd6fd2eb64263c) 

In [ ]:
# initialise camera and bounding box
cap = cv2.VideoCapture(0)
if not video.isOpened():
    print('Error while loading the video!')
    sys.exit()
time.sleep(1.5)
flag, frame = cap.read()
if not flag:
    print('Erro while loading the frame!')
    sys.exit()
bbox = cv2.selectROI(frame)
cv2.destroyAllWindows()
x, y, w, h = bbox
track_window = (x, y, w, h)

In [ ]:
colors = (randint(0, 255), randint(0, 255), randint(0, 255))

In [ ]:
# compute hsv histogram
roi = frame[y:y+h, x:x+w] # RGB -> BGR
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0,180])

In [ ]:
# show histogram
plt.hist(roi.ravel(), 180, [0, 180])
plt.show()

In [ ]:
roi_hist = cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)
parameters = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 1)

In [ ]:
prev_frame_time = 0
new_frame_time = 0
while True:
    flag, frame = cap.read()
    if flag == True:
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0, 180], 1)

        flag, track_window = cv2.CamShift(dst, (x, y, w, h), parameters)
        if flag : 
            pts = cv2.boxPoints(flag)
            pts = np.int0(pts)
            img = cv2.polylines(frame, [pts], True, 255, 2)
        else:
            cv2.putText(frame, 'Tracking failure!', (100,80), cv2.FONT_HERSHEY_SIMPLEX, .75, (0,0,255))
            
        new_frame_time = time.time()
        fps = round(1/(new_frame_time-prev_frame_time),2)
        prev_frame_time = new_frame_time
        cv2.putText(frame,f'Tracker : CamShift, FPS : {fps}HZ', (100, 20), cv2.FONT_HERSHEY_SIMPLEX, .75, (0, 0, 255))

        cv2.imshow('Camshift tracking', img)
        cv2.imshow('Camshift distribution', dst)

        if cv2.waitKey(1) == 13:
            break
    else:
        print('Cannot read webcam')
        break

cv2.destroyAllWindows()
cap.release()

## Optical Flow

Two techniques exists : Sparse and Dense

### Sparse
#### Auto generation of points

In [15]:
# define parameters
parameters_shitomasi = {
    'maxCorners' : 100, # maximal number of corner
    'qualityLevel' : 0.3, # pourcentage of the maximum corner value to be left out
    'minDistance' : 7 # minimal pixel distance between corner
}
parameters_lucas_kanade = {
    'winSize' : (15,15), # minimum size of the pyramid
    'maxLevel' : 2, # number of level for the pyramid
    'criteria' : (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03) # criteria to track
}

In [16]:
colors = np.random.randint(0,255, (100, 3))

In [17]:
video_name = 'walking.avi'

In [18]:
# init video
video = cv2.VideoCapture(os.path.join(paths['VIDEOS_PATH'],video_name))
if not video.isOpened():
    print('Error while loading the video!')
    sys.exit()
flag, frame = video.read()
if not flag:
    print('Erro while loading the frame!')
    sys.exit()

In [19]:
# convert to gray scale
frame_gray_init = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

In [20]:
# create edges list([x,y])
edges = cv2.goodFeaturesToTrack(frame_gray_init, mask = None, **parameters_shitomasi)
mask = np.zeros_like(frame)

In [25]:
# perform tracking
prev_frame_time,new_frame_time = 0,0
while True:
    flag, frame = video.read()
    if not flag:
        print('No more frame to load.')
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    new_edges, status, errors = cv2.calcOpticalFlowPyrLK(frame_gray_init, frame_gray, edges, None, **parameters_lucas_kanade)

    news = new_edges[status == 1]
    olds = edges[status == 1]

    for i, (new, old) in enumerate(zip(news, olds)):
       a, b = new.ravel()
       c, d = old.ravel()

       mask = cv2.line(mask, (int(a),int(b)), (int(c),int(d)), colors[i].tolist(), 2)
       frame = cv2.circle(frame, (int(a),int(b)), 5, colors[i].tolist(), -1)

    img = cv2.add(frame, mask)
    
    new_frame_time = time.time()
    fps = round(1/(new_frame_time-prev_frame_time),2)
    prev_frame_time = new_frame_time
    cv2.putText(img,f'Tracker : OpticalFlow, FPS : {fps}HZ', (100, 20), cv2.FONT_HERSHEY_SIMPLEX, .75, (0, 0, 255))

    cv2.imshow('Optical flow sparse', img)
    if cv2.waitKey(1) == 13: # enter
        break

    frame_gray_init = frame_gray.copy()
    edges = news.reshape(-1,1,2)

cv2.destroyAllWindows()
video.release()

No more frame to load.


#### Select the point

In [26]:
def select_point(event, x, y, flags, params):
    global point, selected_point, old_points
    if event == cv2.EVENT_LBUTTONDOWN:
        point = (x, y)
        selected_point = True
        old_points = np.array([[x, y]], dtype=np.float32)

In [27]:
parameters_lucas_kanade = {
    'winSize' : (15,15), # minimum size of the pyramid
    'maxLevel' : 2, # number of level for the pyramid
    'criteria' : (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03) # criteria to track
}

In [28]:
color_line = (randint(0, 255), randint(0, 255), randint(0, 255))
color_circle = (randint(0, 255), randint(0, 255), randint(0, 255))

In [30]:
video_name = 'walking.avi'

In [31]:
# init video
video = cv2.VideoCapture(os.path.join(paths['VIDEOS_PATH'],video_name))
if not video.isOpened():
    print('Error while loading the video!')
    sys.exit()
flag, frame = video.read()
if not flag:
    print('Erro while loading the frame!')
    sys.exit()

In [32]:
frame_gray_init = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
cv2.namedWindow('Sparse')
cv2.setMouseCallback('Sparse', select_point)

In [33]:
selected_point = False
point = ()
old_points = np.array([[]])
mask = np.zeros_like(frame)

In [35]:
prev_frame_time,new_frame_time = 0,0
while True:
    flag, frame = video.read()
    if not flag:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if selected_point:
        cv2.circle(frame, point, 5, (0, 0, 255), 2)

        new_points, status, errors = cv2.calcOpticalFlowPyrLK(frame_gray_init, frame_gray,
                                                              old_points, None,
                                                              **parameters_lucas_kanade)
        frame_gray_init = frame_gray.copy()
        old_points = new_points

        x, y = new_points.ravel()
        j, k = old_points.ravel()

        mask = cv2.line(mask, (int(x), int(y)), (int(j), int(k)), color_line, 2)
        frame = cv2.circle(frame, (int(x), int(y)), 5, color_circle, -1)

    img = cv2.add(frame, mask)

    new_frame_time = time.time()
    fps = round(1/(new_frame_time-prev_frame_time),2)
    prev_frame_time = new_frame_time
    cv2.putText(img,f'Tracker : OpticalFlow, FPS : {fps}HZ', (100, 20), cv2.FONT_HERSHEY_SIMPLEX, .75, (0, 0, 255))

    cv2.imshow("Sparse", img)
    cv2.imshow("Mask", mask)

    key = cv2.waitKey(1)
    if key == 27: # esc
        break

video.release()
cv2.destroyAllWindows()


### Dense

Useful links:
- [opencv Farneback](https://docs.opencv.org/3.4.15/dc/d6b/group__video__track.html#ga5d10ebbd59fe09c5f650289ec0ece5af)

In [42]:
parameters_farneback = {
    'pyr_scale' : 0.5, # specifying the image scale (<1) to build pyramids
    'levels' : 3, # number of level for the pyramid
    'winsize' : 15, # averaging window size
    'iterations' : 3, # number of iterations the algorithm does
    'poly_n' : 5, # size of the pixel neighborhood
    'poly_sigma' : 1.1, # standard deviation of the Gaussian
    'flags ' : 0, # operation flags
}

In [38]:
video_name = 'walking.avi'

In [46]:
# init video
video = cv2.VideoCapture(os.path.join(paths['VIDEOS_PATH'],video_name))
if not video.isOpened():
    print('Error while loading the video!')
    sys.exit()
flag, first_frame = video.read()
if not flag:
    print('Erro while loading the frame!')
    sys.exit()

In [47]:
frame_gray_init = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
hsv = np.zeros_like(first_frame)
hsv[...,1] = 255 # changing value of the saturation

In [48]:
prev_frame_time,new_frame_time = 0,0
while True:
    flag, frame = video.read()
    if not flag:
        break
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    flow = cv2.calcOpticalFlowFarneback(frame_gray_init, frame_gray, None, **parameters_farneback)
    magnitude, angle = cv2.cartToPolar(flow[...,0], flow[...,1]) # X, Y axis
    hsv[...,0] = angle * (180 / (np.pi / 2)) # modify H channel
    hsv[...,2] = cv2.normalize(magnitude, None, 0, 255, cv2.NORM_MINMAX) # modify V channel

    frame_rgb = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    new_frame_time = time.time()
    fps = round(1/(new_frame_time-prev_frame_time),2)
    prev_frame_time = new_frame_time
    cv2.putText(frame_rgb,f'Tracker : OpticalFlow, FPS : {fps}HZ', (100, 20), cv2.FONT_HERSHEY_SIMPLEX, .75, (0, 0, 255))

    cv2.imshow('Dense optical flow', frame_rgb)
    if cv2.waitKey(1) == 13: # enter
        break

    frame_gray_init = frame_gray

video.release()
cv2.destroyAllWindows()


error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'calcOpticalFlowFarneback'
> Overload resolution failed:
>  - calcOpticalFlowFarneback() missing required argument 'flags' (pos 10)
>  - calcOpticalFlowFarneback() missing required argument 'flags' (pos 10)
